In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/metadata-assistments-dataset/ASSISTments Data Mining Competition 2017_ data description - log data.pdf
/kaggle/input/metadata-assistments-dataset/ASSISTments Data Mining Competition 2017_ data description.xlsx
/kaggle/input/assistments-realdataset/anonymized_full_release_competition_dataset.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_original = pd.read_csv("/kaggle/input/assistments-realdataset/anonymized_full_release_competition_dataset.csv")
df_original.shape

(942816, 82)

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [5]:
# df_original.info

To know the dataset better, below metadata is given in the Assistmen

In [6]:
meta_data = pd.read_excel("/kaggle/input/metadata-assistments-dataset/ASSISTments Data Mining Competition 2017_ data description.xlsx")

In [7]:
meta_data

,Unnamed: 0,Column name,example,description
0,1.0,ITEST_id,8,a deidentified ID/tag used for identifying an individual student
1,2.0,SY ASSISTments Usage,2004-2005,the academic years the student used ASSISTments
2,3.0,AveKnow,0.352416,"average student knowledge level (according to Bayesian Knowledge Tracing algorithm -- cf. Corbett & Anderson, 1995)"
3,4.0,AveCarelessness,0.183276,"average student carelessness (according to San Pedro, Baker, & Rodrigo, 2011 model)"
4,5.0,AveCorrect,0.483902,average student correctness
5,6.0,NumActions,1056,total number of student actions in system
6,7.0,AveResBored,0.208389,"average student affect: boredom (see Pardos, Baker, San Pedro, Gowda, & Gowda, 2014)"
7,8.0,AveResEngcon,0.679126,"average student affect:engaged concentration (see Pardos, Baker, San Pedro, Gowda, & Gowda, 2014)"
8,9.0,AveResConf,0.115905,"average student affect:confusion (see Pardos, Baker, San Pedro, Gowda, & Gowda, 2014)"
9,10.0,AveResFrust,0.112408,"average student affect:frustration (see Pardos, Baker, San Pedro, Gowda, & Gowda, 2014)"


In [8]:
df_original.isnull().any()


studentId                                 False
MiddleSchoolId                            False
InferredGender                             True
SY ASSISTments Usage                      False
AveKnow                                   False
AveCarelessness                           False
AveCorrect                                False
NumActions                                False
AveResBored                               False
AveResEngcon                              False
AveResConf                                False
AveResFrust                               False
AveResOfftask                             False
AveResGaming                              False
action_num                                False
skill                                     False
problemId                                 False
problemType                               False
assignmentId                              False
assistmentId                              False
startTime                               

In [9]:
missing_columns = df_original.columns[df_original.isnull().any()]
print(missing_columns)


Index(['InferredGender', 'sumTime3SDWhen3RowRight', 'isSTEM'], dtype='object')


In [10]:
df_original.columns

Index(['studentId', 'MiddleSchoolId', 'InferredGender', 'SY ASSISTments Usage',
       'AveKnow', 'AveCarelessness', 'AveCorrect', 'NumActions', 'AveResBored',
       'AveResEngcon', 'AveResConf', 'AveResFrust', 'AveResOfftask',
       'AveResGaming', 'action_num', 'skill', 'problemId', 'problemType',
       'assignmentId', 'assistmentId', 'startTime', 'endTime', 'timeTaken',
       'correct', 'original', 'hint', 'hintCount', 'hintTotal', 'scaffold',
       'bottomHint', 'attemptCount', 'frIsHelpRequest', 'frPast5HelpRequest',
       'frPast8HelpRequest', 'stlHintUsed', 'past8BottomOut',
       'totalFrPercentPastWrong', 'totalFrPastWrongCount', 'frPast5WrongCount',
       'frPast8WrongCount', 'totalFrTimeOnSkill', 'timeSinceSkill',
       'frWorkingInSchool', 'totalFrAttempted', 'totalFrSkillOpportunities',
       'responseIsFillIn', 'responseIsChosen', 'endsWithScaffolding',
       'endsWithAutoScaffolding', 'frTimeTakenOnScaffolding',
       'frTotalSkillOpportunitiesScaffolding',
 

In [11]:
print(df_original.dtypes)


studentId                                   int64
MiddleSchoolId                              int64
InferredGender                             object
SY ASSISTments Usage                       object
AveKnow                                   float64
AveCarelessness                           float64
AveCorrect                                float64
NumActions                                  int64
AveResBored                               float64
AveResEngcon                              float64
AveResConf                                float64
AveResFrust                               float64
AveResOfftask                             float64
AveResGaming                              float64
action_num                                  int64
skill                                      object
problemId                                   int64
problemType                                object
assignmentId                                int64
assistmentId                                int64


# Step 2: Data Preparation and Cleaning
The dataset has many columns. We'll select only the features relevant to our task and our target variables (the affective states). Then, we'll handle missing values to ensure our dataset is clean.

In [12]:
# Select the features we previously identified, plus our target variables
features = [
    'studentId',
    'skill',
    'problemId',                                    
    'problemType',
    'assignmentId',
    'timeTaken',
    'correct',
    'bottomHint',
    'attemptCount',
    'original',
    'hintCount',
    'frPast8HelpRequest'
]

# targets = [
#     'average_confidence_frustrated',
#     'average_confidence_confused',
#     'average_confidence_bored',
#     'confidence_gaming'
# ]

targets = [ 'confidence(BORED)',
'confidence(CONCENTRATING)',
'confidence(CONFUSED)',
'confidence(FRUSTRATED)',
'confidence(OFF TASK)',
'confidence(GAMING)' ]



In [13]:
# Create a new DataFrame with only the columns we need
select_df = df_original[features + targets].copy()

# Handle missing values
# We'll drop rows where our target variables are missing, as we can't train on them
select_df.dropna(subset=targets, inplace=True)



print("\nData prepared and cleaned. Shape of the final DataFrame:", select_df.shape)


Data prepared and cleaned. Shape of the final DataFrame: (942816, 18)


In [14]:
select_df.head()

,studentId,skill,problemId,problemType,assignmentId,timeTaken,correct,bottomHint,attemptCount,original,hintCount,frPast8HelpRequest,confidence(BORED),confidence(CONCENTRATING),confidence(CONFUSED),confidence(FRUSTRATED),confidence(OFF TASK),confidence(GAMING)
0,8,properties-of-geometric-figures,1118,textfieldquestion,20405010,49.0,0,0,1,1,1,0,0.597865,0.234294,0.0000,0.0,0.838710,0.008522
1,8,properties-of-geometric-figures,1119,noprobtype,20405010,4.0,1,0,1,0,0,1,0.355694,0.992585,0.9375,0.0,0.600000,0.047821
2,8,sum-of-interior-angles-more-than-3-sides,1120,noprobtype,20405010,6.0,0,0,1,0,0,0,0.355694,0.992585,0.9375,0.0,0.600000,0.047821
3,8,sum-of-interior-angles-more-than-3-sides,1120,noprobtype,20405010,18.0,0,0,2,0,0,0,0.355694,0.617065,0.0000,0.0,0.204082,0.343996
4,8,sum-of-interior-angles-more-than-3-sides,1121,noprobtype,20405010,2.0,1,0,1,0,0,0,0.355694,0.617065,0.0000,0.0,0.204082,0.343996


In [15]:
select_df.dtypes

studentId                      int64
skill                         object
problemId                      int64
problemType                   object
assignmentId                   int64
timeTaken                    float64
correct                        int64
bottomHint                     int64
attemptCount                   int64
original                       int64
hintCount                      int64
frPast8HelpRequest             int64
confidence(BORED)            float64
confidence(CONCENTRATING)    float64
confidence(CONFUSED)         float64
confidence(FRUSTRATED)       float64
confidence(OFF TASK)         float64
confidence(GAMING)           float64
dtype: object

In [16]:
select_df['skill'].unique()

array(['properties-of-geometric-figures',
       'sum-of-interior-angles-more-than-3-sides', 'point-plotting',
       'transformations-rotations', 'reading-graph', 'area', 'perimeter',
       'square-root', 'isosceles-triangle',
       'application: isosceles triangle', 'multiplying-decimals',
       'proportion', 'pythagorean-theorem',
       'interpreting-linear-equations', 'pattern-finding',
       'application: compare points',
       'application: multi-column subtraction',
       'application: simple multiplication',
       'application: compare expressions',
       'application: order of operations',
       'application: multi-column addition', 'noskill',
       'application: read points', 'application: find slope in graph',
       'p-patterns-relations-algebra', 'percent-of', 'venn-diagram',
       'equivalent-fractions-decimals-percents', 'of-means-multiply',
       'fraction-multiplication', 'supplementary-angles', 'transversals',
       'triangle-inequality', 'multiplication

In [17]:
select_df['skill'].nunique()

102

These are the broad classification of skill set:

1️⃣ Geometry & Measurement
2D & 3D Shapes & Properties:
properties-of-geometric-figures, isosceles-triangle, application: isosceles triangle,
supplementary-angles, transversals, triangle-inequality, congruence, similar-triangles,
properties-of-solids

Angles & Theorems:
sum-of-interior-angles-more-than-3-sides, sum-of-interior-angles-triangle,
pythagorean-theorem, meaning-of-pi

Transformations & Graphs:
transformations-rotations, point-plotting, reading-graph, application: read points,
application: compare points, application: find slope in graph, graph interpretation,
graph-shape, interpreting-linear-equations, slope, interpreting-numberline, number-line

Area, Perimeter, Volume:
area, perimeter, area-concept, area-of-circle, surface-area-and-volume,
linear-area-volume-conversion, circumference, measurement, m-measurement,
rate-with-distance-and-time

2️⃣ Algebra & Equations
Expressions & Equations:
making-sense-of-expressions-and-equations, algebraic-manipulation,
substitution, equation-solving, equation-concept, order-of-operations,
application: order of operations, application: compare expressions,
application: simple multiplication, application: multi-column subtraction,
application: multi-column addition, application: finding percentage of a number,
algebra symbolization, symbolization-articulation

Functions & Patterns:
inducing-functions, evaluating-functions, p-patterns-relations-algebra, pattern-finding

Inequalities:
inequalities, inequality-solving

3️⃣ Numbers & Operations
Whole Numbers & Decimals:
addition, subtraction, multiplication, division, simple-calculation,
adding-decimals, subtracting-decimals, multiplying-decimals, divide-decimals,
multiplying-positive-negative-numbers, square-root, exponents, reciprocal,
scientific-notation, rounding

Fractions & Percents:
fractions, fraction-multiplication, fraction-division, comparing-fractions,
equivalent-fractions-decimals-percents, fraction-decimals-percents, reduce-fraction,
finding-percents, percent-of, of-means-multiply, discount, percents

Number Sense:
n-number-sense-operations, ordering-numbers, integers, divisibility, prime-number,
least-common-multiple

4️⃣ Data Handling, Probability & Statistics
Probability & Combinatorics:
probability, combinatorics, venn-diagram

Statistics & Graphs:
mean, median, mode, statistics, circle-graph, stem-and-leaf-plot

5️⃣ Measurement Conversions & Rates
unit-conversion, rate, rate-with-distance-and-time, linear-area-volume-conversion

6️⃣ Miscellaneous
noskill (placeholder)

Some application-based ones could span multiple categories.

In [18]:
select_df['problemType'].unique()

array(['textfieldquestion', 'noprobtype', 'radioquestion', 'other',
       'algebra', 'popupmenuquestion', 'checkboxquestion',
       'interfaceradioquestion1', 'algebrafieldquestion',
       'interfacetextfieldquestion', '1', 'interfaceradioquestion',
       'algebrafieldquestion1', 'interfacepopupmenuquestion1',
       'interfacetextfieldquestion1', '0'], dtype=object)

In [19]:
select_df['problemType'].nunique()

16

# Feature Selection 

In [20]:
easy_skills = [
    # Numbers & Operations
    "addition",
    "subtraction",
    "multiplication",
    "division",
    "adding-decimals",
    "subtracting-decimals",
    "multiplying-decimals",
    "divide-decimals",
    "multiplying-positive-negative-numbers",
    "square-root",
    "exponents",
    "reciprocal",
    "ordering-numbers",
    "rounding",
    "prime-number",
    "least-common-multiple",
    "divisibility",
    "simple-calculation",

    # Fractions & Percents
    "fractions",
    "fraction-multiplication",
    "fraction-division",
    "comparing-fractions",
    "reduce-fraction",
    "equivalent-fractions-decimals-percents",
    "finding-percents",
    "percent-of",
    "of-means-multiply",
    "discount"

    # # Basic Algebra
    # "equation-solving",
    # "equation-concept",
    # "substitution",
    # "order-of-operations",
    # "application: order of operations",
    # "making-sense-of-expressions-and-equations",
    # "algebraic-manipulation",

    # # Basic Statistics
    # "mean",
    # "median",
    # "mode"
]


In [21]:
problem_type_mapping = {
    'textfieldquestion': 'text_input',
    'interfacetextfieldquestion': 'text_input',
    'interfacetextfieldquestion1': 'text_input',
    'algebra': 'algebra_input',
    'algebrafieldquestion': 'algebra_input',
    'algebrafieldquestion1': 'algebra_input',

    'radioquestion': 'multiple_choice',
    'interfaceradioquestion': 'multiple_choice',
    'interfaceradioquestion1': 'multiple_choice',

    'checkboxquestion': 'checkbox',

    'popupmenuquestion': 'dropdown',
    'interfacepopupmenuquestion1': 'dropdown'
}

# Apply mapping to the 'problemType' column only
select_df['problemType'] = select_df['problemType'].map(problem_type_mapping)

# Optional: remove rows where mapping failed (NaN values)
select_df = select_df.dropna(subset=['problemType'])


In [22]:
''' easy_problem_types = [
    "textfieldquestion",
    "radioquestion",
    # "popupmenuquestion",
    "checkboxquestion",
    #"algebra",
    # "algebrafieldquestion",
    "interfaceradioquestion",
    "interfacetextfieldquestion",
    #"algebrafieldquestion1",
    # "interfacepopupmenuquestion1",
    # "interfaceradioquestion1",
    # "interfacetextfieldquestion1"
]'''

easy_problem_types = [
'text_input',
'multiple_choice',
'checkbox',
'multiple_choice',
'text_input'
]

In [23]:
select_df['problemType'].nunique()

5

In [24]:
filtered_df = select_df[
    select_df['skill'].isin(easy_skills) &
    select_df['problemType'].isin(easy_problem_types)
]


In [25]:
filtered_df.shape

(106773, 18)

In [26]:
filtered_df.columns

Index(['studentId', 'skill', 'problemId', 'problemType', 'assignmentId',
       'timeTaken', 'correct', 'bottomHint', 'attemptCount', 'original',
       'hintCount', 'frPast8HelpRequest', 'confidence(BORED)',
       'confidence(CONCENTRATING)', 'confidence(CONFUSED)',
       'confidence(FRUSTRATED)', 'confidence(OFF TASK)', 'confidence(GAMING)'],
      dtype='object')

In [27]:
filtered_df.head()

,studentId,skill,problemId,problemType,assignmentId,timeTaken,correct,bottomHint,attemptCount,original,hintCount,frPast8HelpRequest,confidence(BORED),confidence(CONCENTRATING),confidence(CONFUSED),confidence(FRUSTRATED),confidence(OFF TASK),confidence(GAMING)
29,8,square-root,1093,multiple_choice,20405010,41.0,0,0,1,1,0,0,0.597865,0.374317,0.378151,0.000000,0.222222,0.186970
30,8,square-root,1094,multiple_choice,20405010,4.0,0,0,1,0,0,0,0.355694,0.458033,0.000000,0.000000,0.230769,0.405194
31,8,square-root,1094,multiple_choice,20405010,9.0,1,0,2,0,0,0,0.355694,0.458033,0.000000,0.000000,0.230769,0.405194
32,8,square-root,1095,text_input,20405010,7.0,1,0,1,0,0,1,0.355694,0.399340,0.000000,0.091463,0.204082,0.047821
33,8,square-root,1096,text_input,20405010,21.0,1,0,1,0,0,2,0.355694,0.831852,0.000000,0.091463,0.714286,0.047821


In [28]:
filtered_df['skill'].nunique()

24

In [29]:
filtered_df['problemType'].nunique()

3

In [30]:
filtered_df['assignmentId'].nunique()

167

In [31]:
filtered_df['problemId'].nunique()

358

# -----------------------------------------------------------

In [32]:
# --- Step 2: Define Target and Feature Columns ---

# The target variables are the six confidence scores we want to predict.
target_columns = [ 'confidence(BORED)',
'confidence(CONCENTRATING)',
'confidence(CONFUSED)',
'confidence(FRUSTRATED)',
'confidence(OFF TASK)',
'confidence(GAMING)' ]

# The input features are the columns the model will use to make predictions.
# We exclude the target columns and high-cardinality identifiers like
# studentId, as they don't generalize well.
feature_columns = [
    # 'studentId',
    'skill',
    'problemId',                                    
    'problemType',
    # 'assignmentId',
    'timeTaken',
    'correct',
    'bottomHint',
    'attemptCount',
    'original',
    'hintCount',
    'frPast8HelpRequest'
]


In [33]:
from sklearn.preprocessing import LabelEncoder

# Create label encoders
skill_encoder = LabelEncoder()
problem_type_encoder = LabelEncoder()

# Fit and transform the 'skill' column
filtered_df['skill_encoded'] = skill_encoder.fit_transform(filtered_df['skill'])

# Fit and transform the 'problemType' column
filtered_df['problemType_encoded'] = problem_type_encoder.fit_transform(filtered_df['problemType'])

problem_encoder = LabelEncoder()
filtered_df['problemId_encoded'] = problem_encoder.fit_transform(filtered_df['problemId'])


# Check the result
print(filtered_df[['skill', 'skill_encoded', 'problemType', 'problemType_encoded','problemId', 'problemId_encoded']].head())

# If you want to see the mapping:
print("Skill mapping:", dict(zip(skill_encoder.classes_, skill_encoder.transform(skill_encoder.classes_))))
print()
print("ProblemType mapping:", dict(zip(problem_type_encoder.classes_, problem_type_encoder.transform(problem_type_encoder.classes_))))
print()
print("ProblemType mapping:", dict(zip(problem_encoder.classes_, problem_encoder.transform(problem_encoder.classes_))))


          skill  skill_encoded      problemType  problemType_encoded  problemId  problemId_encoded
29  square-root             21  multiple_choice                    1       1093                200
30  square-root             21  multiple_choice                    1       1094                201
31  square-root             21  multiple_choice                    1       1094                201
32  square-root             21       text_input                    2       1095                202
33  square-root             21       text_input                    2       1096                203
Skill mapping: {'adding-decimals': 0, 'addition': 1, 'comparing-fractions': 2, 'discount': 3, 'division': 4, 'equivalent-fractions-decimals-percents': 5, 'exponents': 6, 'finding-percents': 7, 'fraction-division': 8, 'fraction-multiplication': 9, 'fractions': 10, 'least-common-multiple': 11, 'multiplication': 12, 'multiplying-decimals': 13, 'of-means-multiply': 14, 'ordering-numbers': 15, 'percent-of': 

In [34]:
# Store encoders for later decoding
encoders = {
    'skill': skill_encoder,
    'problemType': problem_type_encoder,
    'problemId': problem_encoder
}

# Example: Decode an encoded skill
decoded_skill = encoders['skill'].inverse_transform([0])  # 0 → actual skill name
print("Decoded skill for 0:", decoded_skill[0])


Decoded skill for 0: adding-decimals


So, we're at a crossroads with the problemId column:

Label Encoding it: This solves the technical problem (it becomes numeric) but creates a severe modeling problem (it encourages overfitting).

Leaving it as raw text: This avoids the overfitting problem, but creates a technical problem (the model can't process it and will error out).

This is why the standard and correct approach is to completely remove the problemId column from the set of features you use for training.

The problemId is not predictive information for the model to learn from; it's metadata for you to keep track of your data. The best practice is to set it as the index of your DataFrame before you create your training data. This keeps the ID associated with each row without including it as a feature for the model.

Here is how you would typically do that:

In [35]:
# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [36]:

# The target variables are the six confidence scores we want to predict.
target_columns = ['confidence(BORED)',
'confidence(CONCENTRATING)',
'confidence(CONFUSED)',
'confidence(FRUSTRATED)',
'confidence(OFF TASK)',
'confidence(GAMING)' ]

feature_columns = [
    'problemId_encoded',
    'problemType_encoded',
    'skill_encoded',
    'timeTaken',
    'correct',
    'bottomHint',
    'attemptCount',
    'original',
    'hintCount',
    'frPast8HelpRequest'
]

In [37]:
filtered_df.shape

(106773, 21)

In [38]:
filtered_df.columns

Index(['studentId', 'skill', 'problemId', 'problemType', 'assignmentId',
       'timeTaken', 'correct', 'bottomHint', 'attemptCount', 'original',
       'hintCount', 'frPast8HelpRequest', 'confidence(BORED)',
       'confidence(CONCENTRATING)', 'confidence(CONFUSED)',
       'confidence(FRUSTRATED)', 'confidence(OFF TASK)', 'confidence(GAMING)',
       'skill_encoded', 'problemType_encoded', 'problemId_encoded'],
      dtype='object')

In [39]:

# --- Step 3: Separate Features (X) and Targets (y) ---

X = filtered_df[feature_columns]
y = filtered_df[target_columns]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42  # Ensures the split is the same every time.
)

# --- You are now ready for modeling! ---

# You can check the shapes of your new datasets to confirm everything worked.
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (85418, 10)
Shape of y_train: (85418, 6)
Shape of X_test: (21355, 10)
Shape of y_test: (21355, 6)


In [40]:
print("Columns in X:")
print(X_train.columns.tolist())


Columns in X:
['problemId_encoded', 'problemType_encoded', 'skill_encoded', 'timeTaken', 'correct', 'bottomHint', 'attemptCount', 'original', 'hintCount', 'frPast8HelpRequest']


# multi-output regression problem

In [41]:
# Create an instance of the RandomForestRegressor model
# random_state is set for reproducibility
model = RandomForestRegressor(n_estimators=1000, random_state=42, n_jobs=-1)

# Train the model on the scaled training data
print("Training the model...")
model.fit(X_train, y_train)
print("Training complete.")

Training the model...
Training complete.


In [42]:
# --- Step 7: Make Predictions on the Test Set ---

# Use the trained model to make predictions on the scaled test data
print("\nMaking predictions on the test set...")
predictions = model.predict(X_test)


Making predictions on the test set...


In [43]:
predictions

array([[0.52041863, 0.52138772, 0.25828204, 0.32605263, 0.40712342,
        0.1813481 ],
       [0.59786477, 0.41717804, 0.32779957, 0.14771374, 0.3660164 ,
        0.2514986 ],
       [0.6809816 , 0.49711735, 0.        , 0.09146342, 0.91666667,
        0.24397085],
       ...,
       [0.59786477, 0.36154736, 0.37985936, 0.00337598, 0.32503566,
        0.20829652],
       [0.35569423, 0.54916569, 0.01833333, 0.1833433 , 0.2304743 ,
        0.18324164],
       [0.6809816 , 0.44384865, 0.        , 0.0956936 , 0.91666667,
        0.23172284]])

In [44]:
# --- Step 8: Evaluate the Model ---

# Calculate the Mean Squared Error (MSE) for the predictions.
# A lower MSE indicates a better fit.
mse = mean_squared_error(y_test, predictions)
print(f"\nOverall Mean Squared Error on the test set: {mse:.4f}")

# You can also look at the Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
print(f"Overall Root Mean Squared Error on the test set: {rmse:.4f}")


Overall Mean Squared Error on the test set: 0.0391
Overall Root Mean Squared Error on the test set: 0.1977


In [45]:
import time
from sklearn.ensemble import RandomForestRegressor

# Create model
model = RandomForestRegressor(n_estimators=1000, random_state=42, n_jobs=-1)

# Measure time
start = time.time()
print("Training the model...")
model.fit(X_train, y_train)
end = time.time()

print("Training complete.")
print(f"Time taken: {end - start:.2f} seconds")


Training the model...


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("Train R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

print("Train RMSE:", mean_squared_error(y_train, y_train_pred, squared=False))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))


analysis of what these numbers mean:

1. Excellent Performance on Training Data
Train R²: 0.87: This is a high R-squared value. It means your model can explain about 87% of the variance in the training data. On the surface, this looks great.

Train RMSE: 0.088: This is a low error value, indicating that the model's predictions on the training data are very close to the actual values.

If we only looked at these training metrics, we would think we have a very strong model.

2. Poor Performance on Test Data
Test R²: 0.44: This R-squared value is significantly lower than the training score. It means the model can only explain 44% of the variance in new, unseen data. This is a major drop in performance.

Test RMSE: 0.184: The error on the test set is more than double the error on the training set (0.184 vs 0.088). This confirms that the model is much less accurate when it encounters data it hasn't seen before.

The Verdict: A Classic Case of Overfitting
This large gap between the training and testing performance is the textbook definition of overfitting.

Your results perfectly confirm the hypothesis from yesterday. Here is what has happened:

The model performed exceptionally well on the training set because it used the problemId_encoded feature to essentially memorize the answers for each specific problem.

When faced with the test set, which contains new problemIds it has never seen before, all that memorized information became useless.

The model was then forced to rely on the actual predictive features, but because it had relied so heavily on the ID shortcut during training, it never learned the true, generalizable patterns. The result is a significant drop in performance.

In short, the model has learned the training data, but it has not learned how to solve the problem in a general way. This model would not be reliable for making predictions on any new data in the future.

In [ ]:
model = RandomForestRegressor(
    n_estimators=500,
    max_depth=20,        # limit depth of trees
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)
# Measure time
start = time.time()
print("Training the model...")
model.fit(X_train, y_train)
end = time.time()

print("Training complete.")
print(f"Time taken: {end - start:.2f} seconds")

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print("Train R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

print("Train RMSE:", mean_squared_error(y_train, y_train_pred, squared=False))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))


In [ ]:
import pandas as pd
pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": model.feature_importances_
}).sort_values(by="Importance", ascending=False)


Observations

timeTaken (41%) → by far the most important feature.

hintCount (15%) and problemId_encoded (10.5%) + correct (10.4%) → also strong predictors.

frPast8HelpRequest, attemptCount, skill_encoded (5–7%) → moderate importance.

original, problemType_encoded, bottomHint (very low importance, <3%) → contribute almost nothing.

# With Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Tighter parameter grid (based on your earlier results)
param_dist = {
    'n_estimators': [300, 500, 700, 1000],
    'max_depth': [15, 20, 25, None],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [2, 4, 5, 8],
    'max_features': ['sqrt', 0.5, 0.7],  # limit features per split
    'bootstrap': [True]
}

# Base model
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Randomized search
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,              # smaller number to keep runtime reasonable
    scoring='r2',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

print("Running RandomizedSearchCV...")
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
print("Best CV Score:", random_search.best_score_)

# Evaluate best model
best_model = random_search.best_estimator_
y_test_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
print("Test R²:", r2_score(y_test, y_test_pred))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))


In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

# Best model from search
best_model = random_search.best_estimator_

# Train predictions
y_train_pred = best_model.predict(X_train)
# Test predictions
y_test_pred = best_model.predict(X_test)

# R²
print("Train R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

# RMSE
print("Train RMSE:", mean_squared_error(y_train, y_train_pred, squared=False))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))


# After dropping original, frPast8HelpRequest. (Assignmentid, problemid,student id)

In [46]:

# The target variables are the six confidence scores we want to predict.
target_columns = ['confidence(BORED)',
'confidence(CONCENTRATING)',
'confidence(CONFUSED)',
'confidence(FRUSTRATED)',
'confidence(OFF TASK)',
'confidence(GAMING)' ]

feature_columns = [
    'problemType_encoded',
    'skill_encoded',
    'timeTaken',
    'correct',
    'bottomHint',
    'attemptCount',
    'hintCount',
]

# Eliminated features are : problemID, original, frPast8HelpRequest.

In [47]:

X = filtered_df[feature_columns]
y = filtered_df[target_columns]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42  # Ensures the split is the same every time.
)

# --- You are now ready for modeling! ---

# You can check the shapes of your new datasets to confirm everything worked.
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (85418, 7)
Shape of y_train: (85418, 6)
Shape of X_test: (21355, 7)
Shape of y_test: (21355, 6)


In [90]:

# X = filtered_df[feature_columns]
# y = filtered_df[target_columns]

# X_train, X_test, y_train, y_test = train_test_split(
#     X,
#     y,
#     test_size=0.2,
#     random_state=42  # Ensures the split is the same every time.
# )

# --- You are now ready for modeling! ---

# You can check the shapes of your new datasets to confirm everything worked.
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)


from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# Tighter parameter grid (based on your earlier results)
param_dist = {
    'n_estimators': [300, 500, 700, 1000],
    'max_depth': [15, 20, 25, None],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [2, 4, 5, 8],
    'max_features': ['sqrt', 0.5, 0.7],  # limit features per split
    'bootstrap': [True]
}

# Base model
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Randomized search
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,              # smaller number to keep runtime reasonable
    scoring='r2',
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

print("Running RandomizedSearchCV...")
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
print("Best CV Score:", random_search.best_score_)

# Evaluate best model
best_model = random_search.best_estimator_
y_test_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score
print("Test R²:", r2_score(y_test, y_test_pred))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))


Shape of X_train: (85418, 7)
Shape of y_train: (85418, 6)
Shape of X_test: (21355, 7)
Shape of y_test: (21355, 6)
Running RandomizedSearchCV...
Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END bootstrap=True, max_depth=20, max_features=0.5, min_samples_leaf=8, min_samples_split=10, n_estimators=700; total time=  51.0s
[CV] END bootstrap=True, max_depth=25, max_features=0.5, min_samples_leaf=2, min_samples_split=15, n_estimators=700; total time=  56.5s
[CV] END bootstrap=True, max_depth=15, max_features=0.7, min_samples_leaf=4, min_samples_split=15, n_estimators=700; total time=  57.3s
[CV] END bootstrap=True, max_depth=25, max_features=0.5, min_samples_leaf=2, min_samples_split=15, n_estimators=700; total time=  55.0s
[CV] END bootstrap=True, max_depth=None, max_features=0.5, min_samples_leaf=4, min_samples_split=10, n_estimators=700; total time=  55.1s
[CV] END bootstrap=True, max_depth=25, max_features=0.7, min_samples_leaf=4, min_samples_split=10, n_estimators=7

In [1]:
from sklearn.metrics import r2_score, mean_squared_error

# Get best estimator from RandomizedSearchCV
best_model = random_search.best_estimator_

# Predictions
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# Train metrics
print("Train R²:", r2_score(y_train, y_train_pred))
print("Train RMSE:", mean_squared_error(y_train, y_train_pred, squared=False))

# Test metrics
print("Test R²:", r2_score(y_test, y_test_pred))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))


NameError: name 'random_search' is not defined

What this means
Best model from recent CV
{
    'n_estimators': 700,
    'min_samples_split': 15,
    'min_samples_leaf': 5,
    'max_features': 0.5,
    'max_depth': 15,
    'bootstrap': True
}


✅ Generalization is good

The gap between Train and Test R² is small (0.52 vs 0.47).

That means your model is not overfitting.

⚖️ # Model is underfitting a bit

Both scores are moderate (around 0.5).

This suggests the model captures some signal but not all.

📊 Error spread is balanced

Train RMSE (0.17) and Test RMSE (0.18) are very close.

That again shows consistency → the model is stable.

# LightGBM is usually faster and more accurate than RandomForest on tabular data.

In [92]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time

# Define base LightGBM model
lgbm = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=-1,           # no limit
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# Wrap for multi-output regression
model = MultiOutputRegressor(lgbm)

# Train
start = time.time()
print("Training LightGBM model...")
model.fit(X_train, y_train)
print(f"Training complete in {time.time() - start:.2f} seconds.")

# Predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Evaluation
print("Train R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))
print("Train RMSE:", mean_squared_error(y_train, y_train_pred, squared=False))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))


Training LightGBM model...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006099 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.433787
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.540525
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000858 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data p

# LGBM hyper parameter tuning

In [51]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time

# --- 1. Validation split (for early stopping later) ---
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Sub-training set size: {X_train_sub.shape}")
print(f"Validation set size: {X_val.shape}")

# --- 2. Reduced hyperparameter grid ---
param_dist = {
    'estimator__n_estimators': [300, 500],   # fewer options
    'estimator__learning_rate': [0.05, 0.1], # common useful values
    'estimator__num_leaves': [31, 63]        # controls complexity
}

# --- 3. Base model ---
base_lgbm = LGBMRegressor(random_state=42, n_jobs=-1)
multi_output_model = MultiOutputRegressor(base_lgbm, n_jobs=-1)

# --- 4. Randomized Search ---
random_search = RandomizedSearchCV(
    estimator=multi_output_model,
    param_distributions=param_dist,
    n_iter=5,    # very small for speed
    cv=2,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\nStarting hyperparameter tuning...")
start_time = time.time()
random_search.fit(X_train_sub, y_train_sub)
end_time = time.time()

print(f"Hyperparameter tuning finished in {(end_time - start_time)/60:.2f} minutes")
print("\nBest parameters found:")
print(random_search.best_params_)

# --- 5. Retrain final model with early stopping ---
best_params_cleaned = {k.replace('estimator__', ''): v for k, v in random_search.best_params_.items()}
final_lgbm = LGBMRegressor(**best_params_cleaned, random_state=42, n_jobs=-1)
final_model = MultiOutputRegressor(final_lgbm, n_jobs=-1)

print("\nTraining final tuned model with early stopping...")
final_model.fit(
    X_train, y_train,
    **{
        "estimator__eval_set": [(X_val, y_val)],
        "estimator__early_stopping_rounds": 50,
        "estimator__verbose": False
    }
)
print("Final model training complete.")

# --- 6. Evaluation ---
y_train_pred_tuned = final_model.predict(X_train)
y_test_pred_tuned = final_model.predict(X_test)

print("\n--- Model Performance ---")
print("Tuned Model Train R²:   ", r2_score(y_train, y_train_pred_tuned))
print("Tuned Model Test  R²:   ", r2_score(y_test, y_test_pred_tuned))
print("Tuned Model Test  RMSE: ", mean_squared_error(y_test, y_test_pred_tuned, squared=False))


Sub-training set size: (68334, 7)
Validation set size: (17084, 7)

Starting hyperparameter tuning...
Fitting 2 folds for each of 5 candidates, totalling 10 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006725 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 339
[LightGBM] [Info] Number of data points in the train set: 34167, number of used features: 7
[LightGBM] [Info] Start training from score 0.541388
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003710 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 339
[LightGBM] [Info] Number of data points in the train set: 34167, number of used features: 7
[LightGBM] [Info] Start training from score 0.140696
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022680 

TypeError: LGBMRegressor.fit() got an unexpected keyword argument 'estimator__eval_set'

In [52]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Best params (drop the "estimator__" prefix)
best_params = {
    "num_leaves": 31,
    "n_estimators": 300,
    "learning_rate": 0.05,
    "random_state": 42,
    "n_jobs": -1,
    "force_row_wise": True   # this avoids overhead warnings
}

print("\nTraining final tuned model...")

# Wrap in MultiOutputRegressor
final_model = MultiOutputRegressor(LGBMRegressor(**best_params))

# Fit (no estimator__ prefix!)
final_model.fit(X_train, y_train)

# Predictions
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

# Evaluation
train_r2 = r2_score(y_train, y_train_pred, multioutput="uniform_average")
test_r2 = r2_score(y_test, y_test_pred, multioutput="uniform_average")

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")



Training final tuned model...
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.433787
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.540525
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.140632
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.146822
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.256492
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the

# for 700 estimators

In [53]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Best params (drop the "estimator__" prefix)
best_params = {
    "num_leaves": 31,
    "n_estimators": 700,
    "learning_rate": 0.05,
    "random_state": 42,
    "n_jobs": -1,
    "force_row_wise": True   # this avoids overhead warnings
}

print("\nTraining final tuned model...")

# Wrap in MultiOutputRegressor
final_model = MultiOutputRegressor(LGBMRegressor(**best_params))

# Fit (no estimator__ prefix!)
final_model.fit(X_train, y_train)

# Predictions
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

# Evaluation
train_r2 = r2_score(y_train, y_train_pred, multioutput="uniform_average")
test_r2 = r2_score(y_test, y_test_pred, multioutput="uniform_average")

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")



Training final tuned model...
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.433787
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.540525
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.140632
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.146822
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.256492
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the

# 1000

In [55]:
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# Best params (drop the "estimator__" prefix)
best_params = {
    "num_leaves": 31,
    "n_estimators": 1000,
    "learning_rate": 0.05,
    "random_state": 42,
    "n_jobs": -1,
    "force_row_wise": True   # this avoids overhead warnings
}

print("\nTraining final tuned model...")

# Wrap in MultiOutputRegressor
final_model = MultiOutputRegressor(LGBMRegressor(**best_params))

# Fit (no estimator__ prefix!)
final_model.fit(X_train, y_train)

# Predictions
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

# Evaluation
train_r2 = r2_score(y_train, y_train_pred, multioutput="uniform_average")
test_r2 = r2_score(y_test, y_test_pred, multioutput="uniform_average")

train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")



Training final tuned model...
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.433787
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.540525
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.140632
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.146822
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the train set: 85418, number of used features: 7
[LightGBM] [Info] Start training from score 0.256492
[LightGBM] [Info] Total Bins 350
[LightGBM] [Info] Number of data points in the

# new

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import time

# Base LightGBM regressor
base_model = lgb.LGBMRegressor(
    objective="regression",
    random_state=42,
    n_jobs=-1
)

# Wrap in MultiOutputRegressor
multi_model = MultiOutputRegressor(base_model, n_jobs=-1)

# Parameter grid (for deeper tuning)
param_grid = {
    "estimator__num_leaves": [15, 31, 63, 127],
    "estimator__max_depth": [-1, 5, 7, 10, 15],
    "estimator__min_child_samples": [5, 10, 20, 30, 50, 100],
    "estimator__subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "estimator__colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
    "estimator__reg_alpha": [0.0, 0.01, 0.1, 1.0, 5.0],
    "estimator__reg_lambda": [0.0, 0.01, 0.1, 1.0, 5.0],
    "estimator__learning_rate": [0.01, 0.05, 0.1],
    "estimator__n_estimators": [200, 400, 600]
}

# Randomized search
search = RandomizedSearchCV(
    multi_model,
    param_distributions=param_grid,
    n_iter=50,                  # number of random combos (increase if time allows)
    scoring="r2",
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Start timing
start_time = time.time()

search.fit(X_train, y_train)

print(f"\nTuning finished in {(time.time()-start_time)/60:.2f} minutes")

# Best params
print("\nBest parameters found:")
print(search.best_params_)

# Final model with best params
final_model = search.best_estimator_

# Evaluate
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

print(f"\nTrain R²: {r2_score(y_train, y_train_pred):.4f}")
print(f"Test R²: {r2_score(y_test, y_test_pred):.4f}")
print(f"Train RMSE: {np.sqrt(mean_squared_error(y_train, y_train_pred)):.4f}")
print(f"Test RMSE: {np.sqrt(mean_squared_error(y_test, y_test_pred)):.4f}")


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020632 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Number of data points in the train set: 56945, number of used features: 7
[LightGBM] [Info] Start training from score 0.147348
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011234 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024830 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 348
[LightGBM] [Info] Number of data points in the train set: 56945, number of used 

In [50]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from lightgbm import LGBMRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time

# --- 1. Validation split (for final retraining, not used in CV) ---
X_train_sub, X_val, y_train_sub, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Sub-training set size: {X_train_sub.shape}")
print(f"Validation set size: {X_val.shape}")

# --- 2. Hyperparameter grid ---
param_dist = {
    'estimator__n_estimators': [200, 500, 800],
    'estimator__learning_rate': [0.01, 0.05, 0.1],
    'estimator__num_leaves': [31, 63],
    'estimator__max_depth': [-1, 10, 20],
    'estimator__subsample': [0.7, 0.9],
    'estimator__colsample_bytree': [0.7, 0.9],
    'estimator__reg_alpha': [0.1, 1.0],
    'estimator__reg_lambda': [0.1, 1.0]
}

# --- 3. Base model and wrapper ---
base_lgbm = LGBMRegressor(random_state=42, n_jobs=-1)
multi_output_model = MultiOutputRegressor(base_lgbm, n_jobs=-1)

random_search = RandomizedSearchCV(
    estimator=multi_output_model,
    param_distributions=param_dist,
    n_iter=15,   # reduced for speed
    cv=2,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\nStarting hyperparameter tuning...")
start_time = time.time()

# No early stopping here
random_search.fit(X_train_sub, y_train_sub)

end_time = time.time()
print(f"Hyperparameter tuning finished in {(end_time - start_time)/60:.2f} minutes")

print("\nBest parameters found:")
print(random_search.best_params_)

# --- 4. Retrain final model with early stopping ---
best_params_cleaned = {k.replace('estimator__', ''): v for k, v in random_search.best_params_.items()}
final_lgbm = LGBMRegressor(**best_params_cleaned, random_state=42, n_jobs=-1)
final_model = MultiOutputRegressor(final_lgbm, n_jobs=-1)

print("\nTraining final tuned model with early stopping...")
final_model.fit(
    X_train, y_train,
    # Now we can use early stopping, because we fit directly
    **{
        "estimator__eval_set": [(X_val, y_val)],
        "estimator__early_stopping_rounds": 50,
        "estimator__verbose": False
    }
)
print("Final model training complete.")

# --- 5. Evaluation ---
y_train_pred_tuned = final_model.predict(X_train)
y_test_pred_tuned = final_model.predict(X_test)

print("\n--- Model Performance ---")
print("Tuned Model Train R²:   ", r2_score(y_train, y_train_pred_tuned))
print("Tuned Model Test  R²:   ", r2_score(y_test, y_test_pred_tuned))
print("Tuned Model Test  RMSE: ", mean_squared_error(y_test, y_test_pred_tuned, squared=False))


Sub-training set size: (68334, 7)
Validation set size: (17084, 7)

Starting hyperparameter tuning...
Fitting 2 folds for each of 15 candidates, totalling 30 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005005 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006314 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006815 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011793 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 339
[LightGBM] [Info] Number of data points in the train set: 34167, number of used features: 7
[LightGBM] [Info] Total Bins 339
[LightGBM] [Info] Number of data

KeyboardInterrupt: 